# WideDeep_TabMlp

## 라이브러리 설치

In [ ]:
# !pip install numpy==1.26.0
# !pip install pandas==2.2.2
# !pip install scikit-learn==1.5.1
# !pip install scipy==1.14.1
# !pip install statsmodels==0.14.2
# !pip install joblib==1.4.2
# !pip install threadpoolctl==3.5.0
# !pip install ipynbname

In [ ]:
# # 1. 기존의 패키지 정리
# !pip uninstall -y torch torchvision torchaudio pytorch-lightning pytorch-tabular

# # 2. 호환 가능한 버전으로 재설치
# !pip install torch==2.0.1+cu118 --index-url https://download.pytorch.org/whl/cu118
# !pip install pytorch-tabular==1.1.1 --no-deps
# !pip install pytorch-lightning==2.0.0

In [ ]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
# !pip install pytorch-widedeep

In [ ]:
# !pip install opencv-python-headless

## import

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import shutil
import ipynbname
import datetime
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, log_loss

from sklearn.utils.class_weight import compute_class_weight


from sklearn.impute import SimpleImputer



import torch

# pytorch-widedeep 라이브러리 import
from pytorch_widedeep import Trainer
from pytorch_widedeep.preprocessing import WidePreprocessor, TabPreprocessor
from pytorch_widedeep.models import Wide, TabMlp, WideDeep
from pytorch_widedeep.metrics import Accuracy,F1Score
from pytorch_widedeep.callbacks import EarlyStopping, ModelCheckpoint



train_path = 'C:/Users/User/Desktop/LG Aimers/LG Aimers 2025/LG Aimers 2025 데이터/train.csv'
test_path = 'C:/Users/User/Desktop/LG Aimers/LG Aimers 2025/LG Aimers 2025 데이터/test.csv'
sample_path = 'C:/Users/User/Desktop/LG Aimers/LG Aimers 2025/LG Aimers 2025 데이터//sample_submission.csv'

## Data Load

In [ ]:
train = pd.read_csv(train_path).drop(columns=["ID"])
test = pd.read_csv(test_path).drop(columns=["ID"])
print(train.shape, test.shape)

## Data Pre-processing

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, FunctionTransformer

def drop_columns(df):
    cols = [
        '불임 원인 - 여성 요인', # 고유값 1
        '불임 원인 - 정자 면역학적 요인' # '1'인 데이터가 train, test에서 모두 1개 >> 신뢰할 수 없음
    ]
    df = df.drop(cols, axis=1)
    return df


    

def 특정시술유형(train, test):
    def categorize_procedure(proc):
        tokens = [token.strip() for token in proc.split(",") if token.strip() and not token.strip().isdigit()]
        # 우선순위에 따른 범주화
        if tokens.count("Unknown") >= 1:
            return "Unknown"
        if tokens.count("AH") >= 1:
            return "AH"
        if tokens.count("BLASTOCYST") >= 1:
            return "BLASTOCYST"
        if tokens.count("ICSI") >= 2 or tokens.count("IVF") >= 2:
            return "2ICSI_2IVF"
        if tokens.count("IVF") >= 1 and tokens.count("ICSI") >= 1:
            return "IVF_ICSI"
        if tokens == "ICSI":
            return "ICSI"
        if tokens == "IVF":
            return "IVF"
        return ",".join(tokens) if tokens else None

    for df in [train, test]:
        df['특정 시술 유형'] = df['특정 시술 유형'].str.replace(" / ", ",")
        df['특정 시술 유형'] = df['특정 시술 유형'].str.replace(":", ",")
        df['특정 시술 유형'] = df['특정 시술 유형'].str.replace(" ", "")

    counts = train['특정 시술 유형'].value_counts()
    allowed_categories = counts[counts >= 100].index.tolist()

    # allowed_categories에 속하지 않는 값은 "Unknown"으로 대체
    train.loc[~train['특정 시술 유형'].isin(allowed_categories), '특정 시술 유형'] = "Unknown"
    test.loc[~test['특정 시술 유형'].isin(allowed_categories), '특정 시술 유형'] = "Unknown"

    train['특정 시술 유형'] = train['특정 시술 유형'].apply(categorize_procedure)
    test['특정 시술 유형'] = test['특정 시술 유형'].apply(categorize_procedure)

    train['시술유형_통합'] = train['시술 유형'].astype(str) + '_' + train['특정 시술 유형'].astype(str)
    test['시술유형_통합'] = test['시술 유형'].astype(str) + '_' + test['특정 시술 유형'].astype(str)

    drop_cols = ['시술 유형', '특정 시술 유형']
    train = train.drop(drop_cols, axis=1)
    test = test.drop(drop_cols, axis=1)

    return train, test

def 시술횟수(df_train):
    for col in [col for col in df_train.columns if '횟수' in col]:
        df_train[col] = df_train[col].replace({'6회 이상':'6회'})
        df_train[col] = df_train[col].str[0].astype(int)
    df_train['시술_임신'] = df_train['총 임신 횟수'] - df_train['총 시술 횟수']
    df_train = df_train.drop('총 시술 횟수', axis=1)
    return df_train

def numeric_process(train, test, cols_to_numeric=None, value='zero'):
    if cols_to_numeric is None:
        cols_to_numeric = [
            '임신 시도 또는 마지막 임신 경과 연수',
            '배란 자극 여부',
            '단일 배아 이식 여부',
            '착상 전 유전 검사 사용 여부',
            '착상 전 유전 진단 사용 여부',
            '총 생성 배아 수',
            '미세주입된 난자 수',
            '미세주입에서 생성된 배아 수',
            '이식된 배아 수',
            '미세주입 배아 이식 수',
            '저장된 배아 수',
            '미세주입 후 저장된 배아 수',
            '해동된 배아 수',
            '해동 난자 수',
            '수집된 신선 난자 수',
            '저장된 신선 난자 수',
            '혼합된 난자 수',
            '파트너 정자와 혼합된 난자 수',
            '기증자 정자와 혼합된 난자 수',
            '동결 배아 사용 여부',
            '신선 배아 사용 여부',
            '기증 배아 사용 여부',
            '대리모 여부',
            'PGD 시술 여부',
            'PGS 시술 여부',
            '난자 채취 경과일',
            '난자 혼합 경과일',
            '배아 이식 경과일',
            '배아 해동 경과일',
        ]
    if value == 'mean':
        imputer_mean = SimpleImputer(strategy='mean')
        train[cols_to_numeric] = imputer_mean.fit_transform(train[cols_to_numeric])
        test[cols_to_numeric] = imputer_mean.transform(test[cols_to_numeric])
    elif value == 'median':
        imputer_median = SimpleImputer(strategy='median')
        train[cols_to_numeric] = imputer_median.fit_transform(train[cols_to_numeric])
        test[cols_to_numeric] = imputer_median.transform(test[cols_to_numeric])
    elif value == 'zero':
        train[cols_to_numeric] = train[cols_to_numeric].fillna(0)
        test[cols_to_numeric] = test[cols_to_numeric].fillna(0)
    else:
        train[cols_to_numeric] = train[cols_to_numeric].fillna(value).astype(float)
        test[cols_to_numeric] = test[cols_to_numeric].fillna(value).astype(float)
    return train, test


def encoding(train, test, seed=42):
    categorical_columns = [
        "시술 시기 코드",
        "시술 당시 나이",
        "배란 유도 유형",
        "배아 생성 주요 이유",

        ## 시술 횟수
        "클리닉 내 총 시술 횟수",
        "IVF 시술 횟수",
        "DI 시술 횟수",

        ## 임신 횟수
        "총 임신 횟수",
        "IVF 임신 횟수",
        "DI 임신 횟수",

        ## 출산 횟수
        "총 출산 횟수",
        "IVF 출산 횟수",
        "DI 출산 횟수",

        "난자 출처",
        "정자 출처",
        "난자 기증자 나이",
        "정자 기증자 나이",

        '시술유형_통합', # 특정시술유형()
    ]
    train[categorical_columns] = train[categorical_columns].astype(str)
    test[categorical_columns] = test[categorical_columns].astype(str)

    train[categorical_columns] = train[categorical_columns].astype('category')
    test[categorical_columns] = test[categorical_columns].astype('category')

    return train, test


def num_feature_scailing(train, test):
    cols_to_divide = [
            '임신 시도 또는 마지막 임신 경과 연수',
            '배란 자극 여부',
            '단일 배아 이식 여부',
            '착상 전 유전 검사 사용 여부',
            '착상 전 유전 진단 사용 여부',
            '총 생성 배아 수',
            '미세주입된 난자 수',
            '미세주입에서 생성된 배아 수',
            '저장된 배아 수',
            '미세주입 후 저장된 배아 수',
            '해동된 배아 수',
            '해동 난자 수',
            '수집된 신선 난자 수',
            '저장된 신선 난자 수',
            '혼합된 난자 수',
            '파트너 정자와 혼합된 난자 수',
            '기증자 정자와 혼합된 난자 수',
            '동결 배아 사용 여부',
            '신선 배아 사용 여부',
            '기증 배아 사용 여부',
            '대리모 여부',
            'PGD 시술 여부',
            'PGS 시술 여부',
        ]
    # # 크기의 차이를 줄이기 위해서서
    # train[cols_to_divide] = train[cols_to_divide] / 10
    # test[cols_to_divide] = test[cols_to_divide] / 10

    cols_to_log = [
            '임신 시도 또는 마지막 임신 경과 연수',
            '배란 자극 여부',
            '단일 배아 이식 여부',
            '착상 전 유전 검사 사용 여부',
            '착상 전 유전 진단 사용 여부',
            '총 생성 배아 수',
            '미세주입된 난자 수',
            '미세주입에서 생성된 배아 수',
            '이식된 배아 수',
            '미세주입 배아 이식 수',
            '저장된 배아 수',
            '미세주입 후 저장된 배아 수',
            '해동된 배아 수',
            '해동 난자 수',
            '수집된 신선 난자 수',
            '저장된 신선 난자 수',
            '혼합된 난자 수',
            '파트너 정자와 혼합된 난자 수',
            '기증자 정자와 혼합된 난자 수',
            '동결 배아 사용 여부',
            '신선 배아 사용 여부',
            '기증 배아 사용 여부',
            '대리모 여부',
            'PGD 시술 여부',
            'PGS 시술 여부',
            '난자 채취 경과일',
            '난자 혼합 경과일',
            '배아 이식 경과일',
            '배아 해동 경과일',
        ]
    log_transformer = FunctionTransformer(np.log1p, validate=True)
    train[cols_to_log] = log_transformer.fit_transform(train[cols_to_log])
    test[cols_to_log] = log_transformer.transform(test[cols_to_log])

    numeric_cols = train.select_dtypes(include=["number"]).columns.tolist()
    cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
    cols_to_scale = [
        col for col in numeric_cols
        if col not in cat_cols and col != '임신 성공 여부'
    ]
    scaler = StandardScaler()
    train[cols_to_scale] = scaler.fit_transform(train[cols_to_scale])
    test[cols_to_scale] = scaler.transform(test[cols_to_scale])

    return train, test





In [ ]:

def all_process(train, test):
    
    train, test = drop_columns(train), drop_columns(test)

    train, test = 특정시술유형(train, test)

    train, test = 시술횟수(train), 시술횟수(test)
    
    # 카테고리 컬럼을 카테고리 타입으로 바꾸기
    train, test = encoding(train, test)

    # 결측값 0으로 채우기기
    train, test = numeric_process(train, test, value='zero')

    # 로그변환 후 정규화화
    train, test = num_feature_scailing(train, test)

    return train, test

train = pd.read_csv(train_path).drop(columns=['ID'])
# test = pd.read_csv(test_path).drop(columns=['ID'])

####### 테스트로 적은 수의 데이터로##################
train = train.sample(n=1000, random_state=42)
###################################################

# train 데이터의 label 컬럼을 기준으로 stratify 설정
train, test = train_test_split(train, test_size=0.2, stratify=train['임신 성공 여부'], random_state=42)


# train, test = all_process(train, test)

print(train.shape, test.shape)

In [ ]:
# 실험 내용
experiment_desc = '''
WideDeep_TabMlp 임신 성공 여부 연속형 변수 임베딩

'''

In [ ]:
import time
start_time = time.time()


# import scipy.special   # 시그모이드 대신 softmax에 사용할 수 있음
# 교차 검증 설정: seed_list를 [333] 하나만 사용, n_splits=3
seed_list = [333]
n_splits = 5

total_auc, total_acc, total_f1 = [], [], []
test_preds = []

# --- DataConfig 설정 ---
# 범주형 변수 지정 (실제 데이터에 맞게 수정)
categorical_cols = [
        "시술 시기 코드",
        "시술 당시 나이",
        "배란 유도 유형",
        "배아 생성 주요 이유",

        ## 시술 횟수
        "클리닉 내 총 시술 횟수",
        "IVF 시술 횟수",
        "DI 시술 횟수",

        ## 임신 횟수
        "총 임신 횟수",
        "IVF 임신 횟수",
        "DI 임신 횟수",

        ## 출산 횟수
        "총 출산 횟수",
        "IVF 출산 횟수",
        "DI 출산 횟수",

        "난자 출처",
        "정자 출처",
        "난자 기증자 나이",
        "정자 기증자 나이",

        '시술유형_통합', # 특정시술유형()
    ]  

# 타겟과 범주형 변수 외 나머지를 연속형 변수로 처리 (실제 컬럼명에 맞게 수정)
continuous_cols = [
            '임신 시도 또는 마지막 임신 경과 연수',
            '배란 자극 여부',
            '단일 배아 이식 여부',
            '착상 전 유전 검사 사용 여부',
            '착상 전 유전 진단 사용 여부',
            '총 생성 배아 수',
            '미세주입된 난자 수',
            '미세주입에서 생성된 배아 수',
            '이식된 배아 수',
            '미세주입 배아 이식 수',
            '저장된 배아 수',
            '미세주입 후 저장된 배아 수',
            '해동된 배아 수',
            '해동 난자 수',
            '수집된 신선 난자 수',
            '저장된 신선 난자 수',
            '혼합된 난자 수',
            '파트너 정자와 혼합된 난자 수',
            '기증자 정자와 혼합된 난자 수',
            '동결 배아 사용 여부',
            '신선 배아 사용 여부',
            '기증 배아 사용 여부',
            '대리모 여부',
            'PGD 시술 여부',
            'PGS 시술 여부',
            '난자 채취 경과일',
            '난자 혼합 경과일',
            '배아 이식 경과일',
            '배아 해동 경과일',
        ]

# Wide 부분: 원-핫 인코딩 (crossed_cols 옵션도 사용할 수 있음)
wide_cols = categorical_cols  # Wide 모델에는 범주형 변수를 원-핫 인코딩으로 처리
crossed_cols = []  # 필요시 두 개 이상의 컬럼을 교차시켜 상호작용 feature 생성

# WidePreprocessor (원-핫 인코딩, crossed_cols 사용 가능)
wide_preprocessor = WidePreprocessor(wide_cols=wide_cols, crossed_cols=crossed_cols)

# Deep 부분: 임베딩 + 연속형 변수 처리
tab_preprocessor = TabPreprocessor(embed_cols=categorical_cols, continuous_cols=continuous_cols)



# 교차 검증 시작
for seed in seed_list:
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    auc_scores, acc_scores, f1_scores = [], [], []
    
    for fold, (train_idx, valid_idx) in enumerate(skf.split(train.drop(columns=['임신 성공 여부']), train["임신 성공 여부"])):
        # Fold 데이터 생성
        fold_train, fold_valid = train.iloc[train_idx], train.iloc[valid_idx]
        fold_train2 = fold_train.copy()
        fold_test = test.copy()  # test 데이터는 별도 사용
        
        # 전처리 
        fold_train, fold_valid = all_process(fold_train, fold_valid)
        _, fold_test = all_process(fold_train2, fold_test)


        # 전처리: 각 Fold 별로 Wide & Deep 데이터 생성
        X_wide_train = wide_preprocessor.fit_transform(fold_train)
        X_wide_valid = wide_preprocessor.transform(fold_valid)
        X_wide_test = wide_preprocessor.transform(fold_test)
        X_tab_train = tab_preprocessor.fit_transform(fold_train)
        X_tab_valid = tab_preprocessor.transform(fold_valid)
        X_tab_test = tab_preprocessor.transform(fold_test)

        # Target 값: 정수형 (0,1)
        y_train = fold_train['임신 성공 여부'].astype(int).values
        y_valid = fold_valid['임신 성공 여부'].astype(int).values

        # Wide 모델: 입력 차원은 원-핫 인코딩된 피처 수####
        wide_model = Wide(input_dim=int(X_wide_train.max().item()), pred_dim=1)


        # Deep 모델: TabMlp 사용
        tab_model = TabMlp(
            column_idx=tab_preprocessor.column_idx,  # 데이터 컬럼의 인덱스 정보 (필수)
            cat_embed_input=tab_preprocessor.cat_embed_input,  # (컬럼명, 고유값 개수, 임베딩 차원) 리스트 (범주형 변수)
            continuous_cols=continuous_cols,  # 연속형 변수 리스트
            
            # ▶ 범주형 임베딩 관련 파라미터 (Categorical Embeddings)
            cat_embed_dropout=0.0,  # 범주형 변수의 임베딩 드롭아웃 확률 (기본값: 0.1)
            use_cat_bias=False,  # 범주형 임베딩에 bias 추가 여부 (기본값: False)
            cat_embed_activation=None,  # 범주형 임베딩의 활성화 함수 ('relu', 'tanh', 'leaky_relu', 'gelu' 가능, 기본값 없음)

            # ▶ 연속형 변수 관련 파라미터 (Continuous Features)
            cont_norm_layer='batchnorm',  # 연속형 변수 정규화 ('batchnorm' 또는 'layernorm', 기본값 없음)
            embed_continuous_method='standard', # 연속형 변수 임베딩 방식
            # 후보 값: 'standard' (기본), 'piecewise' (구간별 변환), 'periodic' (주기적 변환)

            cont_embed_dim=8,  # 연속형 변수 임베딩 차원 (None이면 사용 안 함)
            # 추천: 8~16 정도로 시작하고, 데이터가 복잡할수록 32 이상으로 증가 가능.

            cont_embed_dropout=0.0,  # 연속형 변수 임베딩 드롭아웃 확률 (기본값: 0.0)
            # 0.0 → 드롭아웃 사용 안 함 (데이터가 많거나 과적합 걱정이 없을 때)
            # 0.1~0.3 → 적절한 정규화 효과 (일반적인 경우)
            # 0.4 이상 → 데이터가 매우 적고 노이즈가 심할 때

            cont_embed_activation='relu',  # 연속형 변수 임베딩 활성화 함수 (기본값 없음)
            # ('relu', 'tanh', 'leaky_relu', 'gelu' 가능)

            # ▶ MLP 관련 파라미터 (Multi-Layer Perceptron)
            mlp_hidden_dims=[200, 100],  # MLP의 은닉층 크기 (기본값: [200, 100])
            mlp_activation="relu",  # MLP의 활성화 함수 ('relu', 'tanh', 'leaky_relu', 'gelu' 가능, 기본값: 'relu')
            mlp_dropout=0.1,  # MLP 은닉층 드롭아웃 확률 (기본값: 0.1)
            mlp_batchnorm=True,  # MLP의 배치 정규화 여부 (기본값: True)
            mlp_batchnorm_last=False,  # MLP 마지막 층에도 배치 정규화 적용 여부 (기본값: False)
            mlp_linear_first=True,  # 선형 변환을 먼저 적용할지 여부 (기본값: True)
        )
        
        
        # Wide & Deep 모델 결합
        model = WideDeep(wide=wide_model, deeptabular=tab_model)
        
        # 옵티마이저 및 학습률 스케줄러 설정
        optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)  # Set your desired learning rate here
        

        # ✅ Training 데이터를 딕셔너리 형태로 생성
        X_train = {
            "X_wide": X_wide_train,
            "X_tab": X_tab_train,
            "target": y_train
        }

        # ✅ Validation 데이터를 딕셔너리로 전달 (X_val 방식 사용)
        X_val = {
            "X_wide": X_wide_valid,
            "X_tab": X_tab_valid,
            "target": y_valid
        }
        
        # EarlyStopping 콜백 설정 (patience=5, min_delta=0.001)
        early_stopping = EarlyStopping(
            monitor="val_loss", 
            patience=5, 
            min_delta=0.001, 
            verbose=1
        )
        
        # 현재 날짜/시간을 포함한 파일 이름 생성
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        model_path = f"saved_models/best_model_{timestamp}.pt"
        
        # ✅ 2. ModelCheckpoint: 최상의 모델을 자동 저장
        model_checkpoint = ModelCheckpoint(
            filepath=model_path,  # 모델 저장 경로
            monitor="val_loss",        # 감시할 지표 ('val_loss' 또는 'val_acc')
            save_best_only=True        # 가장 좋은 성능의 모델만 저장
        )
        
        # Trainer 생성: objective "binary"로 설정, 평가 지표로 Accuracy, AUROC, F1Score 사용
        trainer = Trainer(
            model=model, 
            objective="binary", 
            custom_loss_function=None,  # 기본 손실 함수 사용 (필요 시 사용자 정의 함수 가능)
            optimizers=optimizer,       # 옵티마이저 (기본값: Adam)
            initializers=None,          # 가중치 초기화 없음 (기본값)
            callbacks=[early_stopping],  # 조기 종료 (3 에포크 동안 개선 없으면 종료)
            metrics=[Accuracy()],
            verbose=1,                  # 학습 로그 출력 (기본값: 1)
            seed=42                     # 랜덤 시드 설정 (기본값: 1)
        )
        
        # 학습: validation 데이터는 별도의 인자로 전달
        trainer.fit(
            X_train=X_train,  # ✅ Training 데이터를 딕셔너리로 
            n_epochs=100, 
            batch_size=128, 
            X_val=X_val  # ✅ Validation 데이터를 딕셔너리로 전달
        )
        
        
        
        # 모델 학습 후 Validation 예측 코드:
        # 이미 확률이 계산되어 있는 컬럼을 사용합니다.
        valid_probs = trainer.predict_proba(X_wide=X_wide_valid, X_tab=X_tab_valid)[:,1] ####
        valid_pred = (valid_probs > 0.5).astype(int)

        

        # 실제 정답 
        y_valid = fold_valid['임신 성공 여부'].values.astype(int)


        
        # 평가 지표 계산: 클래스 1의 확률 사용
        fold_auc = roc_auc_score(y_valid, valid_probs)
        fold_acc = accuracy_score(y_valid, valid_pred)
        fold_f1 = f1_score(y_valid, valid_pred)
        print(f"Seed[{seed:<3}] Fold {fold + 1} | AUC: {fold_auc:.6f} | Acc: {fold_acc:.6f} | F1: {fold_f1:.6f}")
        
        auc_scores.append(fold_auc)
        acc_scores.append(fold_acc)
        f1_scores.append(fold_f1)
        
        total_auc.append(fold_auc)
        total_acc.append(fold_acc)
        total_f1.append(fold_f1)
        
        # Test 데이터 예측 (각 fold의 모델로 예측한 결과 저장)
        test_pred = trainer.predict_proba(X_wide=X_wide_test, X_tab=X_tab_test)[:,1]
        test_preds.append(test_pred)
    
    # Fold 별 평균 성능 출력
    avg_auc = np.mean(auc_scores)
    avg_acc = np.mean(acc_scores)
    avg_f1 = np.mean(f1_scores)
    
    print("-" * 80)
    print(f"Seed[{seed:<3}] Average Metrics | AUC: {avg_auc:.7f} | Acc: {avg_acc:.7f} | F1: {avg_f1:.7f}")
    print("-" * 80)

# 전체 Validation 평균 성능 출력
val_auc = np.mean(total_auc)
val_acc = np.mean(total_acc)
val_f1 = np.mean(total_f1)
print("-" * 80)
print(f"Validation Average Metrics | AUC: {val_auc:.7f} | Acc: {val_acc:.7f} | F1: {val_f1:.7f}")

finish_time = time.time()
total_time = finish_time - start_time 

print(total_time)

In [ ]:
# stratify_label = train["임신 성공 여부"].astype(str)
old_auc = 0.7289820 * 100
old_acc = 0.7413351 * 100
old_f1 = 0.2031272 * 100

new_auc = val_auc * 100
new_acc = val_acc * 100
new_f1 = val_f1 * 100

def calculate_change(old_value, new_value):
    change = new_value - old_value
    percentage_change = (change / old_value) * 100 if old_value != 0 else float('inf')
    return change, percentage_change

def format_change(change):
    return f"{change:+.6f}"

# 각 지표의 변화량 계산
auc_change, auc_pct = calculate_change(old_auc, new_auc)
acc_change, acc_pct = calculate_change(old_acc, new_acc)
f1_change, f1_pct = calculate_change(old_f1, new_f1)

# 결과 출력
print("\n================= 모델 성능 변화 =================")
print(f"{'Metric':<8}  {'AUC':>12}  {'Acc':>12}  {'F1':>12}")
print("-" * 50)
print(f"{'Old':<8}  {old_auc:>12.6f}  {old_acc:>12.6f}  {old_f1:>12.6f}")
print(f"{'New':<8}  {new_auc:>12.6f}  {new_acc:>12.6f}  {new_f1:>12.6f}")
print(f"{'Change':<8}  {format_change(auc_change):>12}  {format_change(acc_change):>12}  {format_change(f1_change):>12}")
print(f"{'% Change':<8}  {auc_pct:>11.4f}%  {acc_pct:>11.4f}%  {f1_pct:>11.4f}%")
print("=" * 50)

In [ ]:
# 테스트 데이터 활용 지표 계산
# 평가 지표 계산: 클래스 1의 확률 사용

test_auc = roc_auc_score(test['임신 성공 여부'], np.mean(test_preds, axis=0))
print(f"test 데이터 AUC: {test_auc:.6f}")

- 기본 test 데이터 AUC : 0.732737

## Submission

In [ ]:
sample_submission = pd.read_csv(sample_path)
# test_preds
# sample_submission['임신 성공 확률'] = np.mean(test_preds, axis=0)

# ratio = train['임신 성공 여부'].value_counts(normalize=True)[1]
# real_true_count = int(ratio * len(sample_submission))
# print(f'test의 True 갯수: {real_true_count:<5} (추정)')

# count = (sample_submission['임신 성공 확률'] >= 0.5).sum()
# print(f'test의 True 갯수: {count:<5} (예측 결과)')




In [ ]:
now = datetime.datetime.now().strftime("%m%d_%H%M")

In [ ]:
submission_path = 'Submission'
if not os.path.exists(submission_path):
    os.makedirs(submission_path)

code_dir = 'Code'
if not os.path.exists(code_dir):
    os.makedirs(code_dir)


submission_name = f"submission_{now}.csv"
new_notebook_name = f"code_{now}.ipynb"

sample_submission.to_csv(os.path.join(submission_path, submission_name), index=False)


# 현재 노트북 파일 경로 직접 지정 (실제 노트북 파일명으로 수정)
current_notebook = os.path.join(os.getcwd(), "WideDeep_TabMlp_임신 여부.ipynb")

new_notebook_path = os.path.join(code_dir, new_notebook_name)

# 노트북 파일 복사
shutil.copy(current_notebook, new_notebook_path)

print(f"Notebook saved in '{code_dir}' as '{new_notebook_name}'")


In [ ]:
# 📌 SQLite 데이터베이스 설정
db_path = "experiment_results.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 📌 테이블 생성 (처음 실행 시)
cursor.execute('''
CREATE TABLE IF NOT EXISTS experiments (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    code_name TEXT,
    experiment_desc TEXT,
    auc REAL,
    acc REAL,
    f1 REAL
)
''')

In [ ]:
# 데이터 삽입
cursor.execute('''
INSERT INTO experiments (code_name, experiment_desc, auc, acc, f1)
VALUES (?, ?, ?, ?, ?)
''', (new_notebook_name, experiment_desc.strip(), new_auc, new_acc, new_f1))

# 변경사항 저장 & 연결 종료
conn.commit()
conn.close()

print(f"Experiment '{new_notebook_name}' successfully saved in database")

In [ ]:
import sqlite3
import pandas as pd

# SQLite 데이터 조회 함수
def get_experiment_results(db_path="experiment_results.db", num_results=10):
    """
    SQLite 데이터베이스에서 중복된 실험 데이터를 제거하고, 최근 num_results개의 실험 데이터를 불러오는 함수.
    Returns:
        - Pandas DataFrame: 중복 제거된 실험 데이터
    """
    conn = sqlite3.connect(db_path)

    # 중복 제거 & 최신 데이터 선택하는 SQL 쿼리
    query = f"""
    SELECT * FROM experiments
    WHERE id IN (
        SELECT MAX(id)  -- 가장 최신 데이터 선택
        FROM experiments
        GROUP BY code_name -- id 제외하고 중복 판단
    )
    ORDER BY id DESC  -- 최신 데이터부터 정렬
    LIMIT {num_results};
    """

    df = pd.read_sql_query(query, conn)
    conn.close()

    return df

In [ ]:
df_results = get_experiment_results(num_results=100)
df_results.to_csv('experiment_results.csv', index=False, encoding='utf-8-sig', float_format='%.4f')